# Build Docker Image

In this notebook, we build a docker image that we will later use with Azure ML service as custom docker image. We will push the image to Azure Container Registry (ACR).

In [ ]:
from dotenv import get_key, set_key, find_dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

Please fill in the following cell with your information.

In [ ]:
image_name = "YOUR_IMAGE_NAME"  # e.g. torchdet
acr_name = "YOUR_ACR_NAME"

In [ ]:
acr_name = acr_name.replace("-","")
set_key(env_path, "image_name", image_name)

## Create docker file

In [ ]:
%%writefile dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        cmake \
        git \
        curl \
        ca-certificates && \
    rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.7
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION scipy scikit-learn && \
    /opt/conda/bin/conda clean -ya && \
    ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh && \
    echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && \ 
    echo "conda activate py$PYTHON_VERSION" >> ~/.bashrc

ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
        
RUN /opt/conda/bin/conda install --name py$PYTHON_VERSION -c pytorch pytorch=1.1.0 torchvision=0.3.0 cudatoolkit=9.0 \
    matplotlib=3.1.0 Cython=0.29.10
    
# RUN pip install --upgrade pip && \
RUN pip install azureml-sdk[notebooks] && \
    pip install Pillow==6.1 && \
    pip install numpy==1.17 && \
    /opt/conda/bin/conda clean -yt

RUN git clone https://github.com/cocodataset/cocoapi.git && \
    cd cocoapi/PythonAPI && make && cd -

## Create ACR

In this section, we create ACR to register our container image later. Make sure you see "provisioningState": "Succeeded" in the outputs of the following cells before you proceed further.

In [ ]:
acr_resource_group = get_key(env_path, "resource_group" )

In [ ]:
!az acr create --resource-group {acr_resource_group} --name {acr_name} --sku Basic -o table

In [ ]:
acr_server_name = "{}.azurecr.io".format(acr_name)
acr_server_name

In [ ]:
set_key(env_path, "acr_server_name", acr_server_name)

In [ ]:
!az acr login --name {acr_name}

In [ ]:
!az acr update -n {acr_name} --admin-enabled true -o table

## Build image and push to ACR

In this section, we build our image locally and push it to ACR we created earlier.

In [ ]:
image_tag = acr_server_name + "/" + image_name
set_key(env_path, "image_tag", image_tag)

In [ ]:
!docker build -t {image_tag} .

In [ ]:
!docker push {image_tag}

## Save ACR credentials

We will save the ACR credentials and use them in the next notebook.

In [ ]:
acr_username = !az acr credential show -n {acr_name} --query username
acr_username = acr_username[0][1:-1]
set_key(env_path, "acr_username", acr_username)

In [ ]:
acr_password = !az acr credential show -n {acr_name} --query passwords[0].value
acr_password = acr_password[0][1:-1]
set_key(env_path, "acr_password", acr_password)

You can now move on to the next notebook to [tune the hyperparameters of the object detection model](04_TuneHyperparameters.ipynb).